In [ ]:
from plotly import graph_objects as go
from prv_accountant import Accountant
from scipy import stats
import numpy as np
import pandas as pd

In [ ]:
max_compositions = 10000
noise_multiplier = 100.0
sampling_probability = 1.0

In [ ]:
acc_1 = Accountant(noise_multiplier, sampling_probability, 1e-7, max_compositions=max_compositions, eps_error=1.0)
acc_05 = Accountant(noise_multiplier, sampling_probability, 1e-7, max_compositions=max_compositions, eps_error=0.5)
acc_01 = Accountant(noise_multiplier, sampling_probability, 1e-7, max_compositions=max_compositions, eps_error=0.1)

In [ ]:
f_n_01 = acc_01.compute_compositions(max_compositions)
f_n_05 = acc_05.compute_compositions(max_compositions)
f_n_1 = acc_1.compute_compositions(max_compositions)

In [ ]:
def delta_exact(eps):
    mu = np.sqrt(max_compositions)/noise_multiplier
    return stats.norm.cdf(-eps/mu+mu/2)-np.exp(eps)*stats.norm.cdf(-eps/mu-mu/2)

In [ ]:
epss = np.linspace(0.1,5,10)

In [ ]:
data = pd.DataFrame(data={
    "eps": epss,
    "delta_upper-eps_err=0.1":[acc_01.compute_delta_upper(f_n_01, e) for e in epss],
    "delta_lower-eps_err=0.1":[acc_01.compute_delta_lower(f_n_01, e) for e in epss],
    "delta_upper-eps_err=0.5":[acc_05.compute_delta_upper(f_n_05, e) for e in epss],
    "delta_lower-eps_err=0.5":[acc_05.compute_delta_lower(f_n_05, e) for e in epss],
    "delta_upper-eps_err=1.0":[acc_1.compute_delta_upper(f_n_1, e) for e in epss],
    "delta_lower-eps_err=1.0":[acc_1.compute_delta_lower(f_n_1, e) for e in epss],
    "delta-exact": [delta_exact(e) for e in epss]
})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epss, y=data["delta_upper-eps_err=0.1"], name="delta_upper-eps_err=0.1"))
fig.add_trace(go.Scatter(x=epss, y=data["delta_lower-eps_err=0.1"], name="delta_lower-eps_err=0.1"))
fig.add_trace(go.Scatter(x=epss, y=data["delta_upper-eps_err=0.5"], name="delta_upper-eps_err=0.5"))
fig.add_trace(go.Scatter(x=epss, y=data["delta_lower-eps_err=0.5"], name="delta_lower-eps_err=0.5"))
fig.add_trace(go.Scatter(x=epss, y=data["delta_upper-eps_err=1.0"], name="delta_upper-eps_err=1.0"))
fig.add_trace(go.Scatter(x=epss, y=data["delta_lower-eps_err=1.0"], name="delta_lower-eps_err=1.0"))
fig.add_trace(go.Scatter(x=epss, y=data["delta-exact"], name="delta_exact"))
fig.update_yaxes(type="log")
fig.update_layout(width=800, height=500, yaxis={"exponentformat": 'e'})
fig.show()